# Avaliação Contínua 2
## Introdução à Inteligência Artificial, edição 23/24

<img src="Imagens\pacman-naive.png" alt="Drawing" style="width: 200px;"/>

## Introdução
Vamos continuar com o problema do Medo Total da avaliação contínua 1 e vamos procurar a solução do problema usando e desenvolvendo novos algoritmos de procura. Vamos introduzir uma correcção e duas melhorias, uma de eficiência e outra de apresentação. Não iremos descrever o problema aqui de novo, por isso, se está esquecido pode consultar o enunciado da Avaliação Contínua 1.

### Rectificações do modelo em relação à avaliação contínua 1
Considerando o poder P das super-pastilhas, o Medo corrente (MCC) e o objectivo de manter o fantasma amedrontado durante TO instantes
1. Corrigimos o 3º critério da antecipação da falha, quando há medo suficiente para chegar a pelo menos uma pastilha, de acordo com a distância de Manhatan, que passa a ser:

$MC+numPastilhas \times P < TO$, 

em que $minDist$ é a distância de Manhatan do Pacman à pastilha mais próxima.
2. Os testes de antecipação da falha são feitos na geração de cada sucessor e não na expansão dos estados. Só se geram os sucessores de estados onde não se detecte que vai haver uma impossibilidade de solução. Assim, a fronteira cresce menos e o número de expandidos será menor, a memória também.
3. Mudámos o display do estado para mostrar quanto instantes faltam até ao objectivo (TO) e o medo corrente (MC)... Para o display ser perfeito só faltaria apresentar as frequências das visitas a cada célula, mas a perfeição não é por vezes o desejável.

## Formulação como Grafo de Estados
Vamos descrever a nossa formulação.

### Estados
Vamos usar para o estado um `namedtuple` com 5 elementos necessários e suficientes para a representação do estado:

* a posição do Pacman, 
* o conjunto de pastilhas, 
* o tempo que falta para o objectivo, 
* o medo corrente do fantasma e 
* uma tabela com frequência de visita das células da grelha.

Eis a definição do `namedtuple`, que tem de ser importado de `collections` (importamos o nosso `searchPlus` e o `timeit`). 

In [2]:
from collections import namedtuple
from searchPlus import *
import timeit    # Para tirar

EstadoMedo = namedtuple('EstadoMedo', 'pacman, pastilhas, tempo, medo, visitadas')

Como precisamos de usar os algoritmos de procura em grafo, como o custo uniforme e o A*, teremos de redefinir os métodos `__lt__`, `__eq__`e `__hash__`. Criámos assim uma subclasse de `namedtuple`, a que chamámos de `EstadoMedoTotal`.

In [3]:
class EstadoMedoTotal(EstadoMedo):
    """ A classe para representar a informação que muda com as acções.
    um estado é sempre considerado menor do que qualquer outro. """
    def __lt__(self,x):
        return True
    
    def __eq__(self,outro):
        return self.pacman==outro.pacman and self.pastilhas == outro.pastilhas and \
                self.visitadas == outro.visitadas and self.medo == outro.medo and self.tempo == outro.tempo
    
    def __hash__(self):
        return hash(str(self.pacman)+str(self.pastilhas)+str(self.tempo)+str(self.medo)+str(self.visitadas))


Vamos criar o mundo standard, a ser usado por omissão:

In [4]:
parametros="T=26\nM=6\nP=10"
linha1= "= = = = = = = = = =\n"
linha2= "= @ . * . . * . . =\n"
linha3= "= . = = = = = = . =\n"
linha4= "= . = F . . . . . =\n"
linha5= "= . = . . . . . . =\n"
linha6= "= . = . . . . . . =\n"
linha7= "= . = . . . . . . =\n"
linha8= "= * . . . . . . . =\n"
linha9= "= . . . . . . . . =\n"
linha10="= = = = = = = = = =\n"
grelha=linha1+linha2+linha3+linha4+linha5+linha6+linha7+linha8+linha9+linha10
mundoStandard=parametros + "\n" + grelha

```python
parametros="T=26\nM=6\nP=10"
linha1= "= = = = = = = = = =\n"
linha2= "= @ . * . . * . . =\n"
linha3= "= . = = = = = = . =\n"
linha4= "= . = F . . . . . =\n"
linha5= "= . = . . . . . . =\n"
linha6= "= . = . . . . . . =\n"
linha7= "= . = . . . . . . =\n"
linha8= "= * . . . . . . . =\n"
linha9= "= . . . . . . . . =\n"
linha10="= = = = = = = = = =\n"
grelha=linha1+linha2+linha3+linha4+linha5+linha6+linha7+linha8+linha9+linha10
mundoStandard=parametros + "\n" + grelha
```

A função que calcula a distância de Manhatan entre dois pontos $(x,y)$.

In [5]:
# Distância de Manhatan entre 2 pontos
#
def manhatan(p,q):
    (x1,y1) = p
    (x2,y2) = q
    return abs(x1-x2) + abs(y1-y2)

Eis a classe MedoTotal

In [9]:
# A subclasse de Problem: MedoTotal
#
class MedoTotal(Problem):
    """Encontrar um caminho numa grelha 2D com obstáculos. Os obstáculos são células (x, y)."""

    def conv_txt_estado(self,txt):
    
        def processa_linha(dados,y,linha):
            x=0
            for c in linha:
                if c=='*':
                    dados['pastilhas'].add((x,y))
                elif c=='=':
                    dados['obstaculos'].add((x,y))
                elif c=='@':
                    dados['pacman']=(x,y)
                elif c=='F':
                    dados['fantasma']=(x,y)
                if c!= " ":
                    x+=1
        
        linhas=txt.split('\n')
        T=int(linhas[0][2:])
        M=int(linhas[1][2:])
        P=int(linhas[2][2:])
        dim=(len(linhas[3])+1)//2
        
        dados={'dim':dim, 'T':T, 'M':M, 'P':P, 'obstaculos':set(), 'pastilhas':set()}
        y=0
        for l in linhas[3:]:
            processa_linha(dados,y,l)
            y+=1
        return dados
    
        
    def __init__(self, texto_input=mundoStandard):
        diccio=self.conv_txt_estado(texto_input)
        self.initial=EstadoMedoTotal(diccio['pacman'], diccio['pastilhas'], diccio['T'], diccio['M'],{diccio['pacman']:1})
        self.goal=diccio['T'] 
        self.fantasma = diccio['fantasma']
        self.poder = diccio['P']
        self.obstacles=diccio['obstaculos']
        self.dim=diccio['dim']


    directions = {"N":(0, -1), "W":(-1, 0), "E":(1,  0),"S":(0, +1)}  # ortogonais
    
                  
    def result(self, state, action): 
        "Tanto as acções como os estados são representados por pares (x,y)."
        pacman,pastilhas,tempo,medo,visitadas=state
        (x,y) = pacman
        (dx,dy) = self.directions[action]
        npos = (x+dx,y+dy)
        if npos == self.fantasma:
            medo=0
        elif npos in pastilhas:
            pastilhas = pastilhas - {npos}
            medo = self.poder
        else:
            medo -= 1
        tempo -=1
        copia_visitadas = visitadas.copy()
        freq=copia_visitadas.get(npos,0)
        copia_visitadas[npos]=freq+1
        return(EstadoMedoTotal(npos, pastilhas, tempo, medo, copia_visitadas))
    

    # situações de falha antecipada
    #
    def falha_antecipada(self,state):
        if state.tempo <= state.medo:
            return False
        if state.pastilhas == set(): # se não há mais pastilhas e eram necessárias
            return True
        minDist = min(list(map(lambda x: manhatan(state.pacman,x),state.pastilhas)))
        if minDist > state.medo: # se não há tempo (manhatan) para chegar à próxima super-pastilha
            return True
        if (state.medo + self.poder * len(state.pastilhas)) < state.tempo:
            # se o poder de todas as pastilhas mais o medo são insuficientes.
            return True
        return False
    
    def actions(self, state):
        """Podes mover-te para uma célula em qualquer das direcções para uma casa 
           que não seja obstáculo nem fantasma."""
        x, y = state.pacman
        return [act for act in self.directions.keys() 
                if (x+self.directions[act][0],y+self.directions[act][1]) not in (self.obstacles | {self.fantasma}) and 
                not self.falha_antecipada(self.result(state,act))]

    
    def path_cost(self,c,state,action,new):
        return c + new.visitadas[new.pacman]
    
    def goal_test(self,state):
        return state.tempo==0

    def display(self, state):
        """ print the state please"""
        output="Tempo: "+ str(state.tempo) + "\n"
        output+="Medo: "+ str(state.medo)  + "\n"
        for j in range(self.dim):
            for i in range(self.dim):
                if state.pacman ==(i,j):
                    ch = '@'
                elif self.fantasma==(i,j):
                    ch = "F"
                elif (i,j) in self.obstacles:
                    ch = "="
                elif (i,j) in state.pastilhas:
                    ch = '*'
                else:
                    ch = "."
                output += ch + " "
            output += "\n"
        return output  

    def executa(p,estado,accoes,verbose=False):
        """Executa uma sequência de acções a partir do estado devolvendo o triplo formado pelo estado, 
        pelo custo acumulado e pelo booleano que indica se o objectivo foi ou não atingido. Se o objectivo for atingido
        antes da sequência ser atingida, devolve-se o estado e o custo corrente.
        Há o modo verboso e o não verboso, por defeito."""
        custo = 0
        for a in accoes:
            seg = p.result(estado,a)
            custo = p.path_cost(custo,estado,a,seg)
            estado = seg
            objectivo=p.goal_test(estado)
            if verbose:
                p.display(estado)
                print('Custo Total:',custo)
                print('Atingido o objectivo?', objectivo)
            if objectivo:
                break
        return (estado,custo,objectivo)

Podemos confirmar que funciona correctamente para o exemplo seguinte:

In [10]:
parametros="T=24\nM=2\nP=7"
linha1= "= = = = = = = = = =\n"
linha2= "= @ . * . . . . . =\n"
linha3= "= . = = = . = = . =\n"
linha4= "= . = F . * . . * =\n"
linha5= "= . = . . . . . . =\n"
linha6= "= . = . . . . . . =\n"
linha7= "= . = . . . . . . =\n"
linha8= "= . . . . . . . . =\n"
linha9= "= . . . . . * . . =\n"
linha10="= = = = = = = = = =\n"
grelha=linha1+linha2+linha3+linha4+linha5+linha6+linha7+linha8+linha9+linha10
mundoStandard2=parametros + "\n" + grelha
gx=MedoTotal(mundoStandard2)
print('------------     Resolver: \n')
print(gx.display(gx.initial))
start = timeit.default_timer()
resultado= depth_first_graph_search(gx)
if resultado:
    print("Solução Prof-prim (grafo) com custo", str(resultado.path_cost)+":")
    print(resultado.solution())
else:
    print('Sem Solução')
stop = timeit.default_timer()
print('Time: ', stop - start)

------------     Resolver: 

Tempo: 24
Medo: 2
= = = = = = = = = = 
= @ . * . . . . . = 
= . = = = . = = . = 
= . = F . * . . * = 
= . = . . . . . . = 
= . = . . . . . . = 
= . = . . . . . . = 
= . . . . . . . . = 
= . . . . . * . . = 
= = = = = = = = = = 

Solução Prof-prim (grafo) com custo 29:
['E', 'E', 'E', 'E', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'N', 'N', 'N', 'N', 'N', 'S', 'S', 'S', 'S', 'S']
Time:  0.0014910999999955266


Se lhe apresentarmos um problema sem solução, por exemplo:

In [11]:
parametros="T=3\nM=1\nP=7"
linha1= "= = = = = = = = = =\n"
linha2= "= @ . * . . . . . =\n"
linha3= "= . = = = . = = . =\n"
linha4= "= . = F . * . . * =\n"
linha5= "= . = . . . . . . =\n"
linha6= "= . = . . . . . . =\n"
linha7= "= . = . . . . . . =\n"
linha8= "= . . . . . . . . =\n"
linha9= "= . . . . . * . . =\n"
linha10="= = = = = = = = = =\n"
grelha=linha1+linha2+linha3+linha4+linha5+linha6+linha7+linha8+linha9+linha10
mundoStandard2=parametros + "\n" + grelha
gx=MedoTotal(mundoStandard2)
print('------------     Resolver: \n')
print(gx.display(gx.initial))
start = timeit.default_timer()
resultado= depth_first_graph_search(gx)
if resultado:
    print("Solução Prof-prim (grafo) com custo", str(resultado.path_cost)+":")
    print(resultado.solution())
else:
    print('Sem Solução')
stop = timeit.default_timer()
print('Time: ', stop - start)

------------     Resolver: 

Tempo: 3
Medo: 1
= = = = = = = = = = 
= @ . * . . . . . = 
= . = = = . = = . = 
= . = F . * . . * = 
= . = . . . . . . = 
= . = . . . . . . = 
= . = . . . . . . = 
= . . . . . . . . = 
= . . . . . * . . = 
= = = = = = = = = = 

Sem Solução
Time:  0.00013200000000779255


Não encontra solução

O código em cima estará no ficheiro `MedoTotal.py`.

## 3 Perguntas
Vamos ter 3 perguntas nesta avaliação contínua, com base na classe `MedoTotal`, todas de avaliação automática.

### Pergunta 1
Vamos pedir que redefinam a classe `MedoTotal`de modo a utilizarem as distâncias reais do Pacman às pastilhas para a detecção de problema irresolúvel, a que chamámos de antecipação da falha. Ver bloco de notas `pergunta1AvalContinua2.ipynb`.

### Pergunta 2
Construam uma variante da profundidade primeiro, com teste de objectivo na geração dos sucessores, adaptada a problemas em que as soluções estão todas à mesma profundidade, que é o caso deste problema e que permita calcular a melhor solução, podendo devolver informação sobre o número de estados finais, tamanho do espaço de estados, máxima dimensão da fronteira e também com a possibilidade de ser optimizada. Ver bloco de notas `pergunta2AvalContinua2.ipynb`.

### Pergunta 3
Desenvolvam o algoritmo de procura Iterative Deepening A* (`IDA*`) e que o comparem com o A* em termos de estados expandidos e máxima dimensão da fornteira. `pergunta3AvalContinua3.ipynb`

## Submissão

### Quizz
Cada grupo deve completar a implementação da classe pedida e testá-la no link do *quizz* **Avaliação Contínua 2** introduzindo e testando o vosso código. 

Como dissemos atrás, este *quizz* é constituído por 3 perguntas. As 3 perguntas serão avaliadas através de um conjunto de testes automáticos visíveis e mais alguns testes escondidos, valendo no total 1.75 valores (0.55+0.6+0.6)

Podem ir verificando o código (botão check) e submeterem as vezes que quiserem (ambos os elementos do grupo), sendo a submissão com melhor nota a que será considerada.

### Prazo
A submissão fecha às 23:59 de Segunda, 23 de Outubro

### Ficheiro Python
Simultaneamente é necessario submeter o ficheiro Python, que contém todo o código das 3 perguntas, na página da disciplina. Só queremos uma submissão por grupo. Esse ficheiro deve chamar-se *AvalCont2_IIA_23_24_grupoXX.py* em que substituem XX pelo identificador do grupo. 